In [1]:
!pip install stable_baselines3
!pip install gym
!pip install torchvision
!pip install adversarial-robustness-toolbox
!pip install shimmy>=0.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import gym
import numpy as np
import random
from stable_baselines3 import PPO
from art.estimators.classification import PyTorchClassifier
from torchvision import models, transforms
from torchvision.datasets import CIFAR10
from art.defences.preprocessor import FeatureSqueezing, GaussianAugmentation, SpatialSmoothing
from art.attacks.evasion import FastGradientMethod, DeepFool, ElasticNet, UniversalPerturbation, ProjectedGradientDescent
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torch
from gym import spaces
from torchvision.datasets import CIFAR10
from torchvision import transforms
from art.estimators.classification import PyTorchClassifier
import matplotlib.pyplot as plt
from torchvision import models
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

BONUS=0.1

class AttackEnvironment(gym.Env):
    def __init__(self, classifier, dataset):
        super(AttackEnvironment, self).__init__()
        self.attacks = [
            FastGradientMethod,
            DeepFool,
            ElasticNet,
            UniversalPerturbation,
            ProjectedGradientDescent,
        ]
        self.defenses = [
            FeatureSqueezing(bit_depth=5, clip_values=(0.0, 1.0)),
            GaussianAugmentation(sigma=0.1, clip_values=(0.0, 1.0)),
            SpatialSmoothing(window_size=3, clip_values=(0.0, 1.0))
        ]
        self.action_space = spaces.Discrete(len(self.attacks))
        self.observation_space = spaces.Box(low=0, high=1, shape=(3, 32, 32), dtype=np.float32)
        self.classifier = classifier
        self.dataset = dataset
        self.attack_history = []
        self.defense_history=[]
        self.consecutive_no_improvements = 0
        self.max_consecutive_no_improvements = 1  # Adjust this value as needed
        self.target_attack_score = 0.95
        self.max_episode_steps=100
        self.episode_step_count=0
        self.previous_state = None
        self.state = {
            'defects': 0,
        }

    def step(self, action):
        new_state = self.take_action(action)
        attack_class = self.attacks[action]
        attack_name = attack_class.__name__
        attack = attack_class(self.classifier)
        adversarial_sample = attack.generate(self.current_observation)
        attack_score = self.evaluate_attack(adversarial_sample)
        defense_score, defended_sample = self.evaluate_defense(adversarial_sample)
        self.attack_history.append([attack_name, attack_score])
        print("Attack_history",self.attack_history)

        done = self.check_done()
        self.previous_state = self.state.copy()
        self.state['defects'] += defense_score - attack_score
        self.state['defects'] = max(self.state['defects'], 0)
        # Calculate defect differences
        defect_difference, new_defect_difference = self.calculate_defect_differences(
            self.state, self.previous_state, new_state
        )

        # Compute reward
        reward = self.compute_reward(attack_score, defense_score, defect_difference,new_defect_difference)
        self.current_observation = self.get_next_observation()
        return self.current_observation, reward, done, {}

    def take_action(self, action):
        # Instantiate the attack class based on the action
        attack_class = self.attacks[action]
        attack = attack_class(self.classifier)
        # Generate adversarial samples
        adversarial_sample = attack.generate(self.current_observation)
        # Evaluate defense and attack scores
        attack_score = self.evaluate_attack(adversarial_sample)
        defense_score, defended_sample = self.evaluate_defense(adversarial_sample)
        # Update the state based on defense and attack scores
        new_state = self.state.copy()
        new_state['defects'] += defense_score - attack_score
        new_state['defects'] = max(new_state['defects'], 0)
        return new_state

    def evaluate_attack(self, adversarial_sample):
        original_sample = self.current_observation  # Convert to batched input
        original_sample_tensor = torch.tensor(original_sample).float()
        original_output = self.classifier.predict(original_sample)
        adversarial_tensor = torch.tensor(adversarial_sample).float()
        adversarial_output = self.classifier.predict(adversarial_tensor.numpy())

        # Calculate misclassification rate
        misclassification_rate = (original_output.argmax() != adversarial_output.argmax()).mean()
        #print('mis_ratio',misclassification_rate)

        # Calculate perturbation
        perturbation = torch.norm(original_sample_tensor - adversarial_tensor, p=2).mean().item()
        #print('perturbation_attack',perturbation)

        # Calculate confusion matrix
        conf_matrix = confusion_matrix(original_output.argmax(axis=1), adversarial_output.argmax(axis=1))

        # Calculate attack score using the sota_metric
        attack_score = self.sota_metric(misclassification_rate, perturbation, conf_matrix)
        #print('attack_score1',attack_score)
        attack_score = (attack_score).clip(0, 100000)
        #print('attack_score2',attack_score)

        # Update worst_attack_score_so_far and attack history
        if attack_score > self.worst_attack_score_so_far:
            attack_score += BONUS
            self.worst_attack_score_so_far = attack_score
        #print('attack_score3',attack_score)
        #self.attack_history.append(attack_score)

        return attack_score

    def sota_metric(self, misclassification_rate, perturbation, conf_matrix):
        """ Define a custom State-of-the-Art (SOTA) metric """
        # Normalize the confusion matrix
        normalized_conf_matrix = conf_matrix / conf_matrix.sum()

        # Calculate the entropy of the confusion matrix
        confusion_entropy = -np.sum(normalized_conf_matrix * np.log(normalized_conf_matrix + 1e-9))
        #print('çonfusion_entropy',confusion_entropy)

        # Example metric combining misclassification rate, perturbation, and confusion entropy
        # You can adjust these weights and terms to match your specific requirements
        sota_score = misclassification_rate + perturbation - 0.1 * confusion_entropy
        return sota_score

    def evaluate_defense(self, adversarial_sample):
        """Evaluate how effectively the model is defended -> compute a defense score."""
        original_sample = self.current_observation
        original_sample_tensor = torch.tensor(original_sample).float()
        r = random.randint(0,2)
        defense_method = self.defenses[r]
        adversarial_sample_tensor = torch.tensor(adversarial_sample).float()
        defense_method_name = defense_method.__class__.__name__
        defended_sample = defense_method(adversarial_sample_tensor)
        defended_sample = defended_sample[0]
        defended_sample_tensor = torch.tensor(defended_sample).float()
        original_output = self.classifier.predict(original_sample)
        defended_output = self.classifier.predict(defended_sample)
        original_output_tensor = torch.tensor(original_output).float()
        defended_output_tensor = torch.tensor(defended_output).float()
        restoration_rate = (original_output_tensor.argmax() == defended_output_tensor.argmax()).float().mean()
        #print('restoration_rate',restoration_rate)
        perturbation = torch.norm(original_sample_tensor - defended_sample_tensor, p=2).mean().item()
        #print('perturbation_defense',perturbation)
        defense_score = restoration_rate - perturbation
        defense_score = abs(defense_score)
        #print('def_score_1',defense_score)
        defense_score = (defense_score).clip(0, 100000)
        #print('def_score_2',defense_score)

    # Update defense history with defense method's name and score
        self.defense_history.append([defense_method_name, defense_score])
        print("Defense_history", self.defense_history)
        if not hasattr(self, 'best_defense_score_so_far'):
            self.best_defense_score_so_far = -float("inf")

        if defense_score > self.best_defense_score_so_far:
            defense_score += BONUS
            self.best_defense_score_so_far = defense_score
        #print('def_score_3',defense_score)
        return defense_score, defended_sample

    def calculate_defect_differences(self, current_state, previous_state, new_state):
        # Calculate defect differences based on the defect values in current, previous, and new states
        defect_difference = current_state['defects'] - previous_state['defects']
        new_defect_difference = new_state['defects'] - current_state['defects']
        return defect_difference, new_defect_difference

    def check_done(self):
        # Terminate if the attack score surpasses the target or there's no improvement in attack score
        if self.worst_attack_score_so_far > self.target_attack_score:
            return True
        if self.consecutive_no_improvements >= self.max_consecutive_no_improvements:
            return True
        if True:
            self.check_termination_criteria()
        return False

    def calculate_confusion_score(sample, classifier):
        # Convert the sample to a tensor
        sample_tensor = torch.tensor(sample[0]).unsqueeze(0)

        # Get the classifier's predictions as a tensor
        predictions = torch.tensor(classifier.predict(sample_tensor.numpy()))

        # Sort the predictions in descending order
        sorted_predictions, _ = torch.sort(predictions, descending=True)

        # Calculate the difference between the top two predictions and return its absolute value
        confusion_score = torch.abs(sorted_predictions[0] - sorted_predictions[1])

        # Convert the confusion score tensor to a scalar and return
        return confusion_score.item()


    # Sort the dataset by confusion score
    def get_next_observation(self):

        data_by_confusion = sorted(self.dataset,
                                  key=lambda x: torch.sort(
                                  torch.tensor(self.classifier.predict(torch.tensor(x).numpy())),
                                  descending=True
                                  ).values[0, :2].diff().abs().item(),
                                   reverse=True)


        # Take the top 10% of data by confusion
        top_confusing_data = data_by_confusion[:len(self.dataset) // 10]

        # If an attack was particularly successful, prefer similar observations
        if hasattr(self, 'worst_attack_score_so_far') and self.worst_attack_score_so_far > 0.8:
            similarity_threshold = 0.1  # Adjust as needed
            top_confusing_data = [
                x for x in top_confusing_data
                if torch.norm(torch.tensor(x) - torch.tensor(self.current_observation)).float().item() < similarity_threshold
            ]

        # Get the next observation
        observation = top_confusing_data[np.random.randint(0, len(top_confusing_data))]
        return observation

    def compute_reward(self, attack_score, defense_score, defect_difference,new_defect_difference):
        reward = attack_score
        # Subtract defense score from the reward
        reward -= defense_score
        reward = (reward * 100).clip(0, 100)
        # Apply bonus if recent attacks were successful
        last_n_attacks = self.attack_history[-5:]
        #print(last_n_attacks)
        len1 = len(last_n_attacks)
        score_list = []
        for i in range(5):
            if len1 < 5:
                break
            else:
                score_list.append(last_n_attacks[-i][1])
                if all(score > 0.8 for score in score_list):
                    reward += BONUS
                # Apply penalty if recent attacks were unsuccessful
                if all(score < 0.2 for score in score_list):
                    reward -= BONUS
                # Encourage diversity in attack types
        attack_list = []
        for i in range(5):
            if len1 < 5:
                break
            else:
                attack_list.append(last_n_attacks[-i][0])
                recent_attack_types = [action for _, _, _, action in attack_list]
                unique_recent_attacks = len(set(recent_attack_types))
                diversity_bonus = unique_recent_attacks - 1
                reward += diversity_bonus
        # Encourage reduction in defects
        if new_defect_difference<=defect_difference and defect_difference< 0:
            reward += BONUS
        else:
            reward -= BONUS
        #print(reward)
        return reward

    def check_termination_criteria(self):
        self.episode_step_count+=0
        # Episode termination after maximum steps or specific conditions
        if self.episode_step_count >= self.max_episode_steps:
            return True
        if self.worst_attack_score_so_far > 0.90:
            return True
        return False

    def reset(self):
        # Prepare for a new adventure
        self.best_defense_score_so_far = float('-inf')
        self.worst_attack_score_so_far = float('-inf')
        self.current_observation = self.get_next_observation()
        self.attack_history = []
        return self.current_observation

    def render(self, mode='human'):

        # Display information about the worst adversarial attack
        #print('Attack_history:', self.attack_history)
        worst_adversary_index = self.attack_history.index(max(self.attack_history, key=lambda x: x[1]))
        worst_adversary_score = self.attack_history[worst_adversary_index][1]
        worst_adversary_action = worst_adversary_index
        worst_adversary_name = self.attacks[worst_adversary_action].__name__
        # worst_adversary_model = self.model_history[worst_adversary_index]
        print("Worst Adversarial Attack:")
        print(f"Attack Method: {worst_adversary_name}")
        print(f"Attack Score: {worst_adversary_score:.4f}")
        # print(f"Impacted Model: {worst_adversary_model}\n")
        # Display information about the best defense achieved
        #print("Defense_history:", self.defense_history)
        best_defense_index = self.defense_history.index(max(self.defense_history, key=lambda x: x[1]))
        best_defense_score = self.defense_history[best_defense_index][1]
        best_defense_method = self.defense_history[best_defense_index][0]
        # best_defense_model = self.model_history[best_defense_index]
        print("Best Defense Achieved:")
        print(f"Defense Method: {best_defense_method}")
        print(f"Defense Score: {best_defense_score:.4f}")
        # print(f"Model with Best Defense: {best_defense_model}")

# Define transformations for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create train and validation datasets
train_dataset = CIFAR10(root='path/to/cifar10', train=True, download=True, transform=transform)
val_dataset = CIFAR10(root='path/to/cifar10', train=False, download=True, transform=transform)

# Create data loaders to efficiently load the data in batches
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

# Convert the data loaders into arrays
def dataset_to_array(loader):
    data_array = []
    label_array = []
    for batch_data, batch_labels in loader:
        data_array.append(batch_data.numpy())
        label_array.append(batch_labels.numpy())
    #data_array = np.concatenate(data_array)
    #label_array = np.concatenate(label_array)
    return data_array, label_array

train_dataset, train_labels = dataset_to_array(train_loader)
test_dataset, test_labels = dataset_to_array(test_loader)

# Load a pretrained model (example using ResNet-50)
model = models.resnet50(pretrained=False)  # Set pretrained=False
classifier = PyTorchClassifier(
    model=model,
    loss=torch.nn.CrossEntropyLoss(),  # Specify the loss function
    input_shape=(3, 32, 32),  # Adjust input shape based on your data
    nb_classes=1000 , # Adjust the number of classes based on your data
    clip_values=(0, 1)
)
# Create the AttackEnvironment
env = AttackEnvironment(classifier, train_dataset)
# Create a vectorized environment
vec_env = DummyVecEnv([lambda: env])
# Create the PPO agent
agent = PPO("MlpPolicy", vec_env, verbose=1, n_epochs=10, n_steps=2048)
# Training loop
total_timesteps = 10000
for timesteps in range(0, total_timesteps, 2048):
    observation = env.reset()
    episode_reward = 0
    for step in range(2048):
        action, _ = agent.predict(observation, deterministic=True)
        print(action)
        action = action[0]  # Convert the action tensor to a scalar integer
        print("Action taken", action)
        new_observation, reward, done, _ = env.step(action)

        # # Calculate new defect difference
        # _, new_defect_difference = env.calculate_defect_differences(
        #     env.state, env.previous_state, env.take_action(action)
        # )
        # # Adjust the reward based on new_defect_difference
        # reward += new_defect_difference * BONUS
        env.render(mode='human')
        episode_reward += reward
        # agent.collect_rollouts(observation, action, reward, done)
        observation = new_observation
        if done:
            break
    # Train the agent on collected rollouts
    print(f" Total reward in episode {timesteps // 2048}: {episode_reward}")

# Test the trained agent
for episode in range(10):
    observation = env.reset()
    total_reward = 0
    done = False
    while not done:
        action, _ = agent.predict(observation, deterministic=True)
        action = action[0]
        observation, reward, done, _ = env.step(action)
        total_reward += reward
    print(f"Test the trained agent :Episode {episode}: Total Reward = {total_reward}")



/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


100%|██████████| 170498071/170498071 [00:02<00:00, 60740066.32it/s]


Extracting path/to/cifar10/cifar-10-python.tar.gz to path/to/cifar10
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device
[3]
Action taken 3


...

Worst Adversarial Attack:
Attack Method: FastGradientMethod
Attack Score: 53.1115
Best Defense Achieved:
Defense Method: GaussianAugmentation
Defense Score: 74.7103
 Total reward in episode 4: -0.10000000149011612


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Defense_history [['SpatialSmoothing', tensor(42.9516)], ['GaussianAugmentation', tensor(63.4911)], ['SpatialSmoothing', tensor(29.9796)], ['SpatialSmoothing', tensor(29.8796)], ['FeatureSqueezing', tensor(14.4189)], ['SpatialSmoothing', tensor(42.6846)], ['FeatureSqueezing', tensor(14.7163)], ['SpatialSmoothing', tensor(30.9121)], ['FeatureSqueezing', tensor(52.1813)], ['GaussianAugmentation', tensor(74.7103)], ['SpatialSmoothing', tensor(65.0907)]]


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Defense_history [['SpatialSmoothing', tensor(42.9516)], ['GaussianAugmentation', tensor(63.4911)], ['SpatialSmoothing', tensor(29.9796)], ['SpatialSmoothing', tensor(29.8796)], ['FeatureSqueezing', tensor(14.4189)], ['SpatialSmoothing', tensor(42.6846)], ['FeatureSqueezing', tensor(14.7163)], ['SpatialSmoothing', tensor(30.9121)], ['FeatureSqueezing', tensor(52.1813)], ['GaussianAugmentation', tensor(74.7103)], ['SpatialSmoothing', tensor(65.1907)], ['SpatialSmoothing', tensor(65.0907)]]
Attack_history [['ProjectedGradientDescent', 16.134891510109764]]
